Install the required libraries

In [1]:
# Install necessary libraries
!pip install opencv-python mediapipe numpy Pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.15.0 requires protobuf<4.21,>=3.20.3; python_version < "3.11", but you have protobuf 4.25.3 which is incompatible.


Import the libraries

In [2]:
import cv2
import mediapipe as mp
import numpy as np
from PIL import Image
from google.colab.patches import cv2_imshow


Initialize MediaPipe and load virtual clothes image

In [3]:
# Initialize MediaPipe pose solution
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=False, model_complexity=1, enable_segmentation=True, min_detection_confidence=0.5)

# Load virtual clothes image (ensure the image has a transparent background)
clothes_img = Image.open("path to overlay image here")  # Replace with the path to your virtual clothes image


FileNotFoundError: [Errno 2] No such file or directory: 'clothes.png'

Define the function to overlay clothes

In [ ]:
def overlay_clothes(frame, results):
    if results.pose_landmarks:
        # Convert the frame to a PIL image
        frame_pil = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

        # Get the bounding box for the body (assumes clothes cover the torso)
        h, w, _ = frame.shape
        landmarks = results.pose_landmarks.landmark
        x_min = min(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x) * w
        x_max = max(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].x) * w
        y_min = min(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER].y) * h
        y_max = max(landmarks[mp_pose.PoseLandmark.LEFT_HIP].y, landmarks[mp_pose.PoseLandmark.RIGHT_HIP].y) * h

        # Resize clothes image to fit the bounding box
        clothes_resized = clothes_img.resize((int(x_max - x_min), int(y_max - y_min)))

        # Overlay the clothes image onto the frame
        frame_pil.paste(clothes_resized, (int(x_min), int(y_min)), clothes_resized)

        # Convert the frame back to OpenCV format
        frame = cv2.cvtColor(np.array(frame_pil), cv2.COLOR_RGB2BGR)

    return frame


Capture video and apply the overlay in real-time

In [ ]:
# Upload a sample video file to Colab
from google.colab import files
uploaded = files.upload()

# Assuming the uploaded file is named 'video.mp4'
video_path = next(iter(uploaded))

# Capture video from the uploaded file
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to RGB
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame to find pose landmarks
    results = pose.process(frame_rgb)

    # Overlay the virtual clothes
    frame = overlay_clothes(frame, results)

    # Display the frame
    cv2_imshow(frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
